In [31]:
#===============================================================================
# Import libraries
#===============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.cluster import KMeans
%matplotlib inline

In [23]:
df = pd.read_csv('./dataset/players_without_nan.csv')

In [24]:
df['player'] = df['player'].factorize(sort=True)[0]
df['club'] = df['club'].factorize(sort=True)[0]
df['leagueCountry'] = df['leagueCountry'].factorize(sort=True)[0]
df['position'] = df['position'].factorize(sort=True)[0]
df['skintone'] = df['skintone'].apply(lambda x: 0 if x >= 0.5 else 1)
useless_cols = ['playerShort', 'birthday', 'photoID', 'rater1', 'rater2', 'Alpha_3', 'refCount']
df.drop(useless_cols, axis=1, inplace=True)

In [25]:
df_agg = pd.DataFrame()
df_agg['total_yellow'] = df.groupby('player')['yellowCards'].agg('sum')
df_agg['total_yellow_red'] = df.groupby('player')['yellowReds'].agg('sum')
df_agg['total_direct_red'] = df.groupby('player')['redCards'].agg('sum')
df_agg['total_red'] = df.groupby('player')['allreds'].agg('sum')
df_agg['total_game'] = df.groupby('player')['games'].agg('sum')
df_agg['total_win'] = df.groupby('player')['victories'].agg('sum')
df_agg['total_lose'] = df.groupby('player')['defeats'].agg('sum')
df_agg['total_draw'] = df.groupby('player')['ties'].agg('sum')
df_agg['total_goal'] = df.groupby('player')['goals'].agg('sum')

There are some players who transfer to another country. For such players, we consider the country with highest number of occurrence.

In [28]:
df_agg['league'] = df.groupby('player')['leagueCountry'].apply(lambda x: mode(x)[0][0])
df_agg['club'] = df.groupby('player')['club'].apply(lambda x: mode(x)[0][0])
df_agg['height'] = df.groupby('player')['height'].apply(lambda x: mode(x)[0][0])
df_agg['weight'] = df.groupby('player')['weight'].apply(lambda x: mode(x)[0][0])
df_agg['position'] = df.groupby('player')['position'].apply(lambda x: mode(x)[0][0])
df_agg['birthyear'] = df.groupby('player')['birthyear'].apply(lambda x: mode(x)[0][0])
df_agg['skintone'] = df.groupby('player')['skintone'].apply(lambda x: mode(x)[0][0])

In [38]:
kmeans = KMeans(n_clusters=2).fit(df_agg.iloc[:,:-1])
labels = kmeans.labels_

In [39]:
labels

array([1, 1, 0, ..., 0, 0, 1])

In [40]:
df_agg['skintone'].values

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)